In [1]:
import cohere
import json
import io
import warnings

import pandas as pd
from IPython.display import display
from PIL import Image

from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

# APIs

In [2]:
with open('../cohere_api_key.txt', 'r') as f:
    cohere_api_key = f.read()
co = cohere.Client(cohere_api_key)

with open('../stability_api_key.txt', 'r') as f:
    stability_api_key = f.read()
stability_api = client.StabilityInference(
    key=stability_api_key, 
    verbose=True,
)

del cohere_api_key
del stability_api_key

# EDA

In [3]:
with open('../stories/hansel_and_gretel.json') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

<class 'str'> 3286


In [4]:
with open('../stories/red_riding_hood.json') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

<class 'str'> 3645


In [5]:
with open('../stories/three_little_pigs.json', errors='ignore') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

<class 'str'> 3403


# Generating stories

In [6]:
def generate(prompt, model="xlarge", 
             num_generations=5, temperature=0.7, 
             max_tokens=2000, stop_sequences=['<end>']):
             
  prediction = co.generate(
    model=model,
    prompt=prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences=stop_sequences,
    max_tokens=max_tokens,
    temperature=temperature,
    num_generations=num_generations)
  
  # Get list of generations
  gens = []
  likelihoods = []
  for gen in prediction.generations:
      gens.append(gen.text)
      
      sum_likelihood = 0
      for t in gen.token_likelihoods:
          sum_likelihood += t.likelihood
      # Get sum of likelihoods
      likelihoods.append(sum_likelihood)

  pd.options.display.max_colwidth = 200
  # Create a dataframe for the generated sentences and their likelihood scores
  df = pd.DataFrame({'generation':gens, 'likelihood': likelihoods})
  # Drop duplicates
  df = df.drop_duplicates(subset=['generation'])
  # Sort by highest sum likelihood
  df = df.sort_values('likelihood', ascending=False, ignore_index=True)
  
  return df

In [7]:
def generate_image(image_prompt):
  # the object returned is a python generator
  answers = stability_api.generate(
      prompt=image_prompt
  )

  # iterating over the generator produces the api response
  for resp in answers:
      for artifact in resp.artifacts:
          if artifact.finish_reason == generation.FILTER:
              warnings.warn(
                  "Your request activated the API's safety filters and could not be processed."
                  "Please modify the prompt and try again.")
          if artifact.type == generation.ARTIFACT_IMAGE:
              img = Image.open(io.BytesIO(artifact.binary))
              display(img)


In [32]:
result = generate(
    "write a fairy tale",
    max_tokens=1000
)

In [33]:
print(result.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   generation  5 non-null      object 
 1   likelihood  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
None


In [34]:
print(result['generation'][2])

 for adults (well, sort of).
I was intrigued.

It was only the first paragraph, but I was hooked.
Then I was laughing.
Then I was thinking, "Where is this story going?"

I am happy to report that I loved the entire book.
It was such a fun, interesting read.
I read the entire thing in one night.
I didn't want to stop.
I was just too curious to see how the story would end.

The author had a very unique writing style.
I really liked it.
It was simple, yet complex.
It made me want to keep reading.
It made me want to keep turning the pages.

It's not often that I read a book and like the main character so much.
There are just so many good things to say about her.
She is smart, determined, funny, and brave.
I loved the way she saw the world.

I really enjoyed the story.
I am happy to report that it did not end the way I expected.
It had a lot of twists and turns.

There were many funny lines in the book.
The one that made me laugh the most was the following:
"People say love conquers all. Bu